In [1]:
from QuantumCompilation import CircuitState,CircuitShape,CircuitOperations

To build quantum circuits and represent them as quantum states we need a referring ```CircuitShape``` instance.

In [2]:
n_qubits = 4
gates = { 
    1 : [ ["ID"] ],                         #0
    2 : [   ["Sc","St"],                    #1
            ["CZc","CZt"] ],                #2
    3 : [ ["CZc","ID","CZt"] ],             #3
    4 : [ ["CZc","CZc","CZt","CZt"] ]       #4
}

infidelities = [0,0,1e-3,1.5e-3,5e-3] 
crosstalks = [
    [0,0,0,0,0],
    [0,0,0,0,0],
    [0,0,1e-3,0,0],
    [0,0,0,0,0],
    [0,0,0,0,0]
]

shape = CircuitShape(n_qubits,gates,infidelities=infidelities,crosstalks=crosstalks,eq_dir="usage_example/equivalences")
shape.print_steps()

0: ['ID', 'ID', 'ID', 'ID']		 Inf.=0
1: ['ID', 'ID', 'Sc', 'St']		 Inf.=0
2: ['ID', 'ID', 'CZc', 'CZt']		 Inf.=0.001
3: ['ID', 'Sc', 'St', 'ID']		 Inf.=0
4: ['ID', 'CZc', 'CZt', 'ID']		 Inf.=0.001
5: ['ID', 'CZc', 'ID', 'CZt']		 Inf.=0.0015
6: ['Sc', 'St', 'ID', 'ID']		 Inf.=0
7: ['Sc', 'St', 'Sc', 'St']		 Inf.=0
8: ['Sc', 'St', 'CZc', 'CZt']		 Inf.=0.001
9: ['CZc', 'CZt', 'ID', 'ID']		 Inf.=0.001
10: ['CZc', 'CZt', 'Sc', 'St']		 Inf.=0.001
11: ['CZc', 'CZt', 'CZc', 'CZt']		 Inf.=0.003
12: ['CZc', 'ID', 'CZt', 'ID']		 Inf.=0.0015
13: ['CZc', 'CZc', 'CZt', 'CZt']		 Inf.=0.005



"0: ['ID', 'ID', 'ID', 'ID']\t\t Inf.=0\n1: ['ID', 'ID', 'Sc', 'St']\t\t Inf.=0\n2: ['ID', 'ID', 'CZc', 'CZt']\t\t Inf.=0.001\n3: ['ID', 'Sc', 'St', 'ID']\t\t Inf.=0\n4: ['ID', 'CZc', 'CZt', 'ID']\t\t Inf.=0.001\n5: ['ID', 'CZc', 'ID', 'CZt']\t\t Inf.=0.0015\n6: ['Sc', 'St', 'ID', 'ID']\t\t Inf.=0\n7: ['Sc', 'St', 'Sc', 'St']\t\t Inf.=0\n8: ['Sc', 'St', 'CZc', 'CZt']\t\t Inf.=0.001\n9: ['CZc', 'CZt', 'ID', 'ID']\t\t Inf.=0.001\n10: ['CZc', 'CZt', 'Sc', 'St']\t\t Inf.=0.001\n11: ['CZc', 'CZt', 'CZc', 'CZt']\t\t Inf.=0.003\n12: ['CZc', 'ID', 'CZt', 'ID']\t\t Inf.=0.0015\n13: ['CZc', 'CZc', 'CZt', 'CZt']\t\t Inf.=0.005\n"

Now, referring to ```shape```, we can create our circuits using ```CircuitState``` class, which stores them as quantum states.

In [3]:
steps = [
    ['ID','Sc', 'St',"ID"],
    ['CZc','CZc', 'CZt', 'CZt']
]

circuit = CircuitState(shape,steps=steps)
circuit.draw()

q_0: ────■────
         │    
q_1: ─X──┼──■─
      │  │  │ 
q_2: ─X──■──┼─
            │ 
q_3: ───────■─

To simulate the compilation, we need to find the equivalences that will be encoded in the driving Hamiltonian.

In [ ]:
dmax = 2
shape.find_equivalences(dmax) #saves files containing the equivalences in the directory specified to the constructor

The simulation is handled by ```CircuiOperation``` class, which, given a shape, calculates all the creation and destruction operators.

In [6]:
op = CircuitOperations(shape)

n_steps=2
op.load_H(dmax,n_steps) #load driving and infidelity Hamiltonians as attributes of op

Loaded the two Hamiltonians we are ready to compile ```circuit```.

In [7]:
compiled_circuit = op.compile(circuit)

compiled_circuit.measure().draw()

q_0: ─■────
      │    
q_1: ─■──X─
         │ 
q_2: ─■──X─
      │    
q_3: ─■────

In [13]:
print("Initial infidelity:",CircuitOperations.expected_value(op.get_Hi(n_steps),circuit))
print("Final infidelity:",CircuitOperations.expected_value(op.get_Hi(n_steps),compiled_circuit))
print()
print("Is the final circuit equivalent to the inital one?", circuit.equivalent(compiled_circuit))

Initial infidelity: 0.005
Final infidelity: 0.003

Is the final circuit equivalent to the inital one? True
